In [28]:
import os
import requests
import numpy as np
import matplotlib as plt
import pandas as pd
import json
import glob
import folium
import polyline
from dotenv import load_dotenv

# Load the environment variable containing the OneMap token
load_dotenv('../key.env')
TOKEN = os.getenv('ONEMAPTOKEN')
api_key = os.getenv("API_KEY")

%run get_bus_info_function.ipynb
%run get_geospatial_function.ipynb
bus_services_df = get_bus_info("https://datamall2.mytransport.sg/ltaodataservice/BusServices", api_key)
bus_routes_df = get_bus_info("https://datamall2.mytransport.sg/ltaodataservice/BusRoutes", api_key)
bus_stops_df = get_bus_info("https://datamall2.mytransport.sg/ltaodataservice/BusStops", api_key)
geospatial_train_path = "../datasets/geospatial_layer/TrainStation_Jul2024/RapidTransitSystemStation.shp"
train_stations = pd.read_excel("../datasets/Train_Stations.xls")
geospatial_train_gdf = gpd.read_file(geospatial_train_path)

In [18]:
bus_routes_df.head()

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309


In [19]:
bus_stops_df.head()

,BusStopCode,RoadName,Description,Latitude,Longitude
0,01012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
1,01013,Victoria St,St. Joseph's Ch,1.297710,103.853225
2,01019,Victoria St,Bras Basah Cplx,1.296990,103.853022
3,01029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414
4,01039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491


In [20]:
bus_routes_stops = pd.merge(bus_routes_df, bus_stops_df, on = "BusStopCode", how = 'left')

In [25]:
bus_routes_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25548 entries, 0 to 25547
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ServiceNo     25548 non-null  object 
 1   Operator      25548 non-null  object 
 2   Direction     25548 non-null  int64  
 3   StopSequence  25548 non-null  int64  
 4   BusStopCode   25548 non-null  object 
 5   Distance      25548 non-null  float64
 6   WD_FirstBus   25548 non-null  object 
 7   WD_LastBus    25548 non-null  object 
 8   SAT_FirstBus  25548 non-null  object 
 9   SAT_LastBus   25548 non-null  object 
 10  SUN_FirstBus  25548 non-null  object 
 11  SUN_LastBus   25548 non-null  object 
 12  RoadName      25548 non-null  object 
 13  Description   25548 non-null  object 
 14  Latitude      25548 non-null  float64
 15  Longitude     25548 non-null  float64
dtypes: float64(3), int64(2), object(11)
memory usage: 3.1+ MB


### Calling Routing API from OneMapSg

In [24]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json
import time as time_module  # Renaming to avoid conflict with variable names

# Load the environment variable containing the OneMap token
load_dotenv('../key.env')
TOKEN = os.getenv('ONEMAPTOKEN')

# Function to fetch bus route from OneMap API
def fetch_route(start, end, date, route_time, mode='BUS'):
    url = 'https://www.onemap.gov.sg/api/public/routingsvc/route'
    params = {
        'start': start,
        'end': end,
        'routeType': 'pt',
        'date': date,
        'time': route_time,
        'mode': mode,
        'maxWalkDistance': 1000,
        'numItineraries': 3
    }
    headers = {
        'Authorization': f'Bearer {TOKEN}'
    }
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching route from {start} to {end}: {e}")
        return None

# Main function to iterate through the bus routes dataframe and fetch the route
def main(bus_routes_df):
    # Group bus routes by service number and direction
    grouped = bus_routes_df.groupby(['ServiceNo', 'Direction'])

    for (service, direction), group in grouped:
        print(f"Processing Bus Service: {service}, Direction: {direction}")

        # Extract the list of bus stops in the direction of the current service
        stop_codes = group['BusStopCode'].tolist()

        # Iterate through stop pairs to create start and end points
        for i in range(len(stop_codes) - 1):
            start_code = stop_codes[i]
            end_code = stop_codes[i + 1]

            # Get the coordinates of start and end bus stops from bus stops dataframe
            start = f"{group.iloc[i]['Latitude']},{group.iloc[i]['Longitude']}"
            end = f"{group.iloc[i + 1]['Latitude']},{group.iloc[i + 1]['Longitude']}"

            date = '10-29-2024'  # Replace with actual date
            route_time = '14:00:00'    # Replace with actual time

            # Fetch route from OneMap
            data = fetch_route(start, end, date, route_time)
            if data is None:
                continue

            # Save route to file if route data is fetched
            file_path = f"../datasets/routes/onemapsg/{service}_{direction}_{i}.json"
            with open(file_path, 'w') as f:
                json.dump(data, f, indent=2)
            print(f"Generated {file_path}")

            # Delay to avoid API rate limiting
            time_module.sleep(1)

# Use the actual merged DataFrame
main(bus_routes_stops)


Processing Bus Service: 10, Direction: 1
Generated ../datasets/routes/onemapsg/10_1_0.json
Generated ../datasets/routes/onemapsg/10_1_1.json
Generated ../datasets/routes/onemapsg/10_1_2.json
Generated ../datasets/routes/onemapsg/10_1_3.json
Generated ../datasets/routes/onemapsg/10_1_4.json
Generated ../datasets/routes/onemapsg/10_1_5.json
Generated ../datasets/routes/onemapsg/10_1_6.json
Generated ../datasets/routes/onemapsg/10_1_7.json
Generated ../datasets/routes/onemapsg/10_1_8.json
Generated ../datasets/routes/onemapsg/10_1_9.json
Generated ../datasets/routes/onemapsg/10_1_10.json
Generated ../datasets/routes/onemapsg/10_1_11.json
Generated ../datasets/routes/onemapsg/10_1_12.json
Generated ../datasets/routes/onemapsg/10_1_13.json
Generated ../datasets/routes/onemapsg/10_1_14.json
Generated ../datasets/routes/onemapsg/10_1_15.json
Generated ../datasets/routes/onemapsg/10_1_16.json
Generated ../datasets/routes/onemapsg/10_1_17.json
Generated ../datasets/routes/onemapsg/10_1_18.json


In [30]:
bus_routes_stops = pd.merge(bus_routes_df, bus_stops_df, on = "BusStopCode", how = 'left')
#only considering trunk services
bus_routes_stops = bus_routes_stops.merge(
    bus_services_df[['ServiceNo', 'Category']], 
    on='ServiceNo',  # Merge on BusStopCode
    how='left'  # Use 'left' join to keep all rows from bus_routes_stops
)

### Visualising 

In [31]:

# Define the path to your JSON files for bus service 10 and 67, for both directions 1 and 2
files_10_direction_1 = sorted(glob.glob('../datasets/routes/onemapsg/10_1_*.json'))
files_10_direction_2 = sorted(glob.glob('../datasets/routes/onemapsg/10_2_*.json'))
files_67_direction_1 = sorted(glob.glob('../datasets/routes/onemapsg/82_1_*.json'))
files_67_direction_2 = sorted(glob.glob('../datasets/routes/onemapsg/82_2_*.json'))

# Initialize a Folium map centered at an approximate central point
m = folium.Map(location=[1.354, 103.943], zoom_start=13, tiles='CartoDB positron')

# Function to add a slight offset to coordinates
def offset_coordinates(coords, lat_offset=0.0, lon_offset=0.0):
    return [(lat + lat_offset, lon + lon_offset) for lat, lon in coords]

# Helper function to add polylines to the map with optional offset and color
def add_route_to_map(files, color, lat_offset=0.0, lon_offset=0.0):
    for file in files:
        with open(file, 'r') as f:
            data = json.load(f)

        # Extract the itineraries from the plan, using only the first itinerary to avoid excessive overlapping
        if "plan" in data and "itineraries" in data["plan"]:
            itinerary = data["plan"]["itineraries"][0]  # Only use the first itinerary
            all_coords = []
            for leg in itinerary['legs']:
                if 'legGeometry' in leg and 'points' in leg['legGeometry']:
                    # Decode the polyline to get coordinates
                    decoded_coords = polyline.decode(leg['legGeometry']['points'])
                    all_coords.extend(decoded_coords)

            # Apply offset to the coordinates if specified
            offset_coords = offset_coordinates(all_coords, lat_offset=lat_offset, lon_offset=lon_offset)

            # Add the merged polyline to the Folium map
            folium.PolyLine(offset_coords, color=color, weight=2.5, opacity=0.7).add_to(m)

# Add routes for bus service 10, direction 1 (with negative latitude offset)
add_route_to_map(files_10_direction_1, color="blue", lat_offset=-0.00005)

# Add routes for bus service 10, direction 2 (with positive latitude offset)
add_route_to_map(files_10_direction_2, color="red", lat_offset=0.00005)

# Add routes for bus service 67, direction 1 (with no offset)
add_route_to_map(files_67_direction_1, color="green")

# Add routes for bus service 67, direction 2 (with positive latitude offset)
add_route_to_map(files_67_direction_2, color="orange", lat_offset=0.00005)

# Display the map
m


In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json
import time as time_module  # Renaming to avoid conflict with variable names

# Load the environment variable containing the OneMap token
load_dotenv('../key.env')
TOKEN = os.getenv('ONEMAPTOKEN')

# Load train station information
df_mrt_stations = pd.DataFrame({
    "Station_Code": ["NS1", "NS2", "NS3", "NS4", "NS5"],
    "MRT_Station": ["Jurong East", "Bukit Batok", "Bukit Gombak", "Choa Chu Kang", "Yew Tee"],
    "MRT_Line": ["North-South Line", "North-South Line", "North-South Line", "North-South Line", "North-South Line"],
    "Longitude": [103.742263, 103.749541, 103.751910, 103.744369, 103.747402],
    "Latitude": [1.333209, 1.348997, 1.358672, 1.385172, 1.397550]
})

# Function to fetch rail route from OneMap API
def fetch_route(start, end, date, route_time, mode='BUS'):
    url = 'https://www.onemap.gov.sg/api/public/routingsvc/route'
    params = {
        'start': start,
        'end': end,
        'routeType': 'pt',
        'date': date,
        'time': route_time,
        'mode': mode,
        'maxWalkDistance': 1000,
        'numItineraries': 3
    }
    headers = {
        'Authorization': f'Bearer {TOKEN}'
    }
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching route from {start} to {end}: {e}")
        return None

# Main function to iterate through MRT stations dataframe and fetch the route
def main(mrt_stations_df):
    # Iterate through each station to create routes between consecutive stations
    for i in range(len(mrt_stations_df) - 1):
        start_station = mrt_stations_df.iloc[i]
        end_station = mrt_stations_df.iloc[i + 1]

        start = f"{start_station['Latitude']},{start_station['Longitude']}"
        end = f"{end_station['Latitude']},{end_station['Longitude']}"

        date = '10-29-2024'  # Replace with actual date
        route_time = '12:00:00'  # Set initial time to 12:00 PM

        # Attempt fetching the route, with retries if needed
        for _ in range(5):  # Attempt up to 5 times with increasing time intervals
            data = fetch_route(start, end, date, route_time)
            if data is not None:
                break
            # Increment the time by 30 minutes for the next attempt
            hour, minute, second = map(int, route_time.split(':'))
            minute += 15
            if minute >= 60:
                minute -= 60
                hour += 1
            route_time = f"{hour:02d}:{minute:02d}:{second:02d}"

        # Save route to file if route data is fetched
        if data is not None:
            file_path = f"../datasets/routes/onemapsg_mrt/{start_station['Station_Code']}.json"
            with open(file_path, 'w') as f:
                json.dump(data, f, indent=2)
            print(f"Generated {file_path}")

            # Delay to avoid API rate limiting
            time_module.sleep(1)
        else:
            print(f"Failed to fetch route between {start_station['MRT_Station']} and {end_station['MRT_Station']}")

# Run the main function
main(df_mrt_stations)
